# Westeros Part 2a.ii - calculation emission taxes

In the previous tutorial (``westeros_emissions_bounds.ipynb``), we showed how to introduce emissions into a stylized energy systems model, and what happens if you put a constraint on total CO2 emissions.

Now, we will tackle the complementary policy to emissions constraints, namely emissions taxes.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully
- You have run the tutorial on introducing emissions (``westeros_emissions_bounds.ipynb``)

In [1]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [2]:
mp = ixmp.Platform()

In [3]:
model = 'Westeros Electrified'
base = message_ix.Scenario(mp, model=model, scenario='baseline')

In [4]:
scen_bd = message_ix.Scenario(mp, model=model, scenario='emission_bound')

In [5]:
emissions = scen_bd.var('EMISS', {'node': 'Westeros'})
emissions

,node,emission,type_tec,year,lvl,mrg
0,Westeros,CO2,all,700,520.801378,0.0
1,Westeros,CO2,all,710,888.705879,0.0
2,Westeros,CO2,all,720,840.492743,0.0


In [6]:
emission_prices = scen_bd.var('PRICE_EMISSION')
emission_prices

,node,type_emission,type_tec,year,lvl,mrg
0,Westeros,GHG,all,700,11.507833,0.0
1,Westeros,GHG,all,710,18.745048,0.0
2,Westeros,GHG,all,720,30.533708,0.0


When setting a cumlulative bound, the undiscounted price of emission is the same in different model years (see the marginals of equation EMISSION_CONSTRAINT). However, considering the year-to-year discount factor, we observe an ascending trend in emission prices shown in PRICE_EMISSION above. This means the emission price in later years is higher as the value of money in the future is lower compared to today. 

## Setting an emissions tax instead of a bound

Again, we choose the emissions prices from the first example (with a cumulative bound) as the tax level over time.

In [7]:
scen_tax = base.clone(model, 'carbon_tax','introducing a carbon tax',
                      keep_solution=False)
scen_tax.check_out()

In [8]:
scen_tax.add_set('emission', 'CO2')
scen_tax.add_cat('emission', 'GHG', 'CO2')
scen_tax.add_par('emission_factor', scen_bd.par('emission_factor'))

In [9]:
scen_tax.add_set('type_year', [700, 710, 720])

base_tax_emission = {
    'node': 'Westeros',
    'type_year': [700, 710, 720],
    'type_tec': 'all',
    'unit': 'USD/tCO2',
    'type_emission': 'GHG',
    'value': emission_prices.set_index('year').lvl
}

# in case you need to add the new unit to the platform
mp.add_unit('USD/tCO2')

tax_emission = make_df(base_tax_emission)
scen_tax.add_par('tax_emission', tax_emission)
scen_tax.par('tax_emission')

INFO:root:unit `USD/tCO2` is already defined in the platform instance


,node,type_emission,type_tec,type_year,value,unit
0,Westeros,GHG,all,700,11.507833,USD/tCO2
1,Westeros,GHG,all,710,18.745048,USD/tCO2
2,Westeros,GHG,all,720,30.533708,USD/tCO2


These values are taken as a “sweet spot” of prices on carbon such that coal, wind and gas are used
at the same time.

In [10]:
scen_tax.commit(comment='setting taxes on emissions')

In [11]:
scen_tax.solve()

In [12]:
scen_tax.var('EMISS', {'node': 'Westeros'})

,node,emission,type_tec,year,lvl,mrg
0,Westeros,CO2,all,700,520.801378,0.0
1,Westeros,CO2,all,710,888.705879,0.0
2,Westeros,CO2,all,720,830.772151,0.0


## Close the connection to the database

In [13]:
mp.close_db()